In [62]:
import numpy as np
import pandas as pd
from sklearn.utils.extmath import randomized_svd
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")

class FA():
    """
    Clase para análisis exploratorio de los Factores de una matriz de datos mediante método de Factores Principales.

    Esta clase:
        (1) Entrena un modelo de análisis factorial mediante el Método de Factores Principales ("Principal Factor Method"), según se define en Rencher (2002, chap. 13);
        (2) Devuelve: eigenvalues, eigenvectors, loading matrix, communalities, uniquenesses, correlation matrix;
        (3) Permite al generación de una rotación de las cargas ("orthogonal varimax rotation").

    Parámetros
    ----------
    X : np.ndarray
        Matriz de datos.
    n_factors : int, opcional
        Número de factores a extraer.
        Default es None.
    svd_method : {‘randomized’, 'np.svd'}
        Método para la realización de la descomposicón en valores singulares ("Singular Value Decomposition").
        Default es 'randomized', de librería scikit-learn. Para cualquier otra opción, SVD será realizada desde numpy.

    Atributos
    ---------
    get_eig: :obj:`numpy.ndarray`
        Matrices de autovalores y autovectores.
    get_loadings: :obj:`numpy.ndarray`
        Matriz de cargas.
    get_communalities_uniquenesses: :obj:`numpy.ndarray`
        Mtrices de comunalidades y unicidades.
    get_fa: :obj: personalizado
        Salida similiar a STATA.
    get_comparative_corr_matrix: :obj: personalizado
        Comparación para la matriz de correlaciones (estimada vs. original).
    get_rotated_loadings: :obj:`numpy.ndarray`
        Cargas rotadas. Permite ajuste Kaiser (None por Default).

    """
    def __init__(self, X, n_factors=None, svd_method="randomized"):
        self.X = X
        self.n_factors = n_factors
        self.svd_method = svd_method

    def get_eig(self):
        """ 
        Regresa los autovalores y autovectores de la matriz de correlación corregida según Método de Factores Principales,
        de acuerdo al método de SVD seleccionado. 
        """ 

        self.X_corr = np.corrcoef(self.X, rowvar=False)
        self.X_corr_inv = np.linalg.inv(self.X_corr)
        self.X_corr_inv_diag = np.diag(self.X_corr_inv)
        self.R = 1 - (1 / self.X_corr_inv_diag)
        self.X_corr_o = self.X_corr.copy()
        np.fill_diagonal(self.X_corr, self.R)
        self.eigenvalues, _ = np.linalg.eig(self.X_corr) #Para conocer los autovalores negativos
        if self.svd_method == "randomized_svd":
            self.eigenvectors, self.eigenvalues_p, _ = randomized_svd(self.X_corr, n_components=self.X_corr.shape[1], random_state=1234567890)
        else:
            self.eigenvectors, self.eigenvalues_p, _ = np.linalg.svd(self.X_corr, full_matrices=False)

        return self.eigenvalues, self.eigenvectors

    def get_loadings(self):
        """ 
        Regresa las cargas de los Factores, conservando solo aquellas cargas cuyos autovalores sean positivos.
        """  

        self.get_eig()

        self.loadings = self.eigenvectors * np.sqrt(self.eigenvalues_p)
        positive_filter = np.array([i for i, value in enumerate(np.round(self.eigenvalues_p, decimals=10)) if value in np.round(self.eigenvalues, decimals=10)])
        self.positive_loadings = self.loadings[:, positive_filter]

        if self.n_factors == None:
            self.positive_loadings
        else:
            self.positive_loadings = self.positive_loadings[:, :self.n_factors]

        return self.positive_loadings
    
    def get_communalities_uniquenesses(self):
        """ 
        Regresa las comunalidades y unicidades de los Factores.
        """ 

        self.get_loadings()
        self.communalities, self.uniquenesses = np.sum((self.positive_loadings**2), axis=1), 1 - np.sum((self.positive_loadings**2), axis=1)

        return self.communalities, self.uniquenesses
        
    
    def get_fa(self):
        """
        Regresa un resumen detallado de análisis factorial, replicando salida de STATA.
        """      

        self.get_communalities_uniquenesses()                 
        loadings_matrix = pd.DataFrame(self.positive_loadings, columns=[f"Factor {i+1}" for i in range(self.positive_loadings.shape[1])], index=[f"x{i+1}" for i in range(self.positive_loadings.shape[0])]).reset_index(names='Variable')
        loadings_matrix = pd.concat([loadings_matrix, pd.DataFrame(self.uniquenesses, columns=['Uniqueness']), pd.DataFrame(self.communalities, columns=['Communality'])], axis=1)
        eigenv_matrix = pd.DataFrame(np.sort(self.eigenvalues)[::-1], columns=['Eigenvalue'], index=[f"Factor {i+1}" for i in range(self.eigenvalues.shape[0])]).reset_index(names='Factor')
        eigenv_proportion = [np.sort(self.eigenvalues)[::-1][i] / np.sum(np.sort(self.eigenvalues)[::-1]) for i in range(len(self.eigenvalues))]
        eigenv_proportion_cs = np.cumsum(eigenv_proportion)
        eigenv_matrix = pd.concat([eigenv_matrix, pd.DataFrame(eigenv_proportion, columns=['Proportion']), pd.DataFrame(eigenv_proportion_cs, columns=['Cumulative'])], axis=1)

        eigenv_table = tabulate(eigenv_matrix, headers='keys', tablefmt="fancy_grid", showindex=False, floatfmt=".4f")
        loadings_table = tabulate(loadings_matrix, headers='keys', tablefmt="fancy_grid", showindex=False, floatfmt=".4f")

        eigenv_lines = eigenv_table.splitlines()
        loadings_lines = loadings_table.splitlines()

        max_lines = max(len(eigenv_lines), len(loadings_lines))
        eigenv_lines += [''] * (max_lines - len(eigenv_lines))
        loadings_lines += [''] * (max_lines - len(loadings_lines))

        combined_output = "\n".join(f"{eigenv: <40}   {loading}" for eigenv, loading in zip(eigenv_lines, loadings_lines))
        
        print("Resultados de Factor Analysis")
        print(combined_output)
    
    def get_comparative_corr_matrix(self):
        """
        Regresa una comparación de la matriz de correlaciones (estimada por el modelo factorial vs. original).
        """
        self.get_communalities_uniquenesses()
        estimated_corr = self.positive_loadings @ self.positive_loadings.T + np.diag(self.uniquenesses)
        estimated_corr_matrix = pd.DataFrame(estimated_corr, columns=[f"x{i+1}" for i in range(estimated_corr.shape[1])], index=[f"x{i+1}" for i in range(estimated_corr.shape[0])]).reset_index(names='Variables')
        corr = self.X_corr_o
        corr_matrix = pd.DataFrame(corr, columns=[f"x{i+1}" for i in range(corr.shape[1])], index=[f"x{i+1}" for i in range(corr.shape[0])]).reset_index(names='Variables')

        est_corr_table = tabulate(estimated_corr_matrix, headers='keys', tablefmt="fancy_grid", showindex=False, floatfmt=".4f")
        corr_table = tabulate(corr_matrix, headers='keys', tablefmt="fancy_grid", showindex=False, floatfmt=".4f")

        est_corr_lines = est_corr_table.splitlines()
        corr_lines = corr_table.splitlines()

        max_lines = max(len(est_corr_lines), len(corr_lines))
        est_corr_lines += [''] * (max_lines - len(est_corr_lines))
        corr_lines += [''] * (max_lines - len(corr_lines))

        combined_output = "\n".join(f"{eigenv: <40}   {loading}" for eigenv, loading in zip(est_corr_lines, corr_lines))
     

        print("Matriz de correlación estimada (izquierda) vs. Matriz de correlación (derecha)")
        print(combined_output)
        

    def get_rotated_loadings(self, kaiser=False):
        """
        Regresa la cargas rotadas del modelo factorial (orthogonal varimax rotation").
        Permite aplicar adecuación Kaiser.
        """
        self.get_communalities_uniquenesses()
        X = self.positive_loadings.copy()
        n_rows, n_cols = X.shape
        if n_cols < 2:
            return X

        # normalize the loadings matrix
        # using sqrt of the sum of squares (Kaiser)
        if kaiser:
            normalized_mtx = np.apply_along_axis(
                lambda x: np.sqrt(np.sum(x**2)), 1, X.copy()
            )
            X = (X.T / normalized_mtx).T

        # initialize the rotation matrix
        # to N x N identity matrix
        rotation_mtx = np.eye(n_cols)

        d = 0
        for _ in range(5000):
            old_d = d

            # take inner product of loading matrix
            # and rotation matrix
            basis = np.dot(X, rotation_mtx)

            # transform data for singular value decomposition using updated formula :
            # B <- t(x) %*% (z^3 - z %*% diag(drop(rep(1, p) %*% z^2))/p)
            diagonal = np.diag(np.squeeze(np.repeat(1, n_rows).dot(basis**2)))
            transformed = X.T.dot(basis**3 - basis.dot(diagonal) / n_rows)

            # perform SVD on
            # the transformed matrix
            U, S, V = np.linalg.svd(transformed)

            # take inner product of U and V, and sum of S
            rotation_mtx = np.dot(U, V)
            d = np.sum(S)

            # check convergence
            if d < old_d * (1 + 1e-5):
                break

        # take inner product of loading matrix
        # and rotation matrix
        X = np.dot(X, rotation_mtx)

        # de-normalize the data
        if kaiser:
            X = X.T * normalized_mtx
        else:
            X = X.T

        # convert loadings matrix to data frame
        rotated_loadings = X.T.copy()
        return rotated_loadings

In [63]:
final = pd.read_excel(r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\MECMT04\TP AEM - database.xlsx")

final.columns = [f"X{i}" for i in range(len(final.columns))]
final = final.iloc[:,2:].values

In [64]:
fa = FA(final)
pd.DataFrame(fa.get_rotated_loadings())

,0,1,2,3
0,0.054495,-0.734506,0.139047,0.084986
1,-0.197313,-0.783663,-0.228598,0.024281
2,0.208433,0.330184,-0.471200,-0.298080
3,0.037655,-0.173268,-0.709790,0.067278
4,0.875801,0.124088,-0.071768,-0.188519
5,0.879000,0.002539,0.000013,0.170965
6,0.063678,-0.158350,-0.265593,0.370071


In [9]:
fa.get_estimated_corr_matrix()

          0         1         2         3         4         5         6
0  1.000000  0.535130 -0.322015  0.036342 -0.069417  0.060567  0.114300
1  0.535130  1.000000 -0.199402  0.292244 -0.258222 -0.171280  0.181228
2 -0.322015 -0.199402  1.000000  0.265037  0.313529  0.133084 -0.024176
3  0.036342  0.292244  0.265037  1.000000  0.049735  0.044152  0.243248
4 -0.069417 -0.258222  0.313529  0.049735  1.000000  0.737913 -0.014584
5  0.060567 -0.171280  0.133084  0.044152  0.737913  1.000000  0.118837
6  0.114300  0.181228 -0.024176  0.243248 -0.014584  0.118837  1.000000           0         1             2         3             4             5  \
0  0.000000  0.071771  1.411969e-02 -0.058105  9.864752e-03  6.853152e-03   
1  0.071771  0.000000 -3.544898e-02  0.081438  5.884523e-03 -2.295594e-02   
2  0.014120 -0.035449 -1.110223e-16  0.045048  4.192798e-02 -4.048848e-02   
3 -0.058105  0.081438  4.504802e-02  0.000000 -7.879195e-03  1.927687e-02   
4  0.009865  0.005885  4.192798e-02 -0.

In [13]:
fa.get_rotated_loadings()

(          0         1         2         3
 0  0.054495  0.734506 -0.139047  0.084986
 1 -0.197313  0.783663  0.228598  0.024281
 2  0.208433 -0.330184  0.471200 -0.298080
 3  0.037655  0.173268  0.709790  0.067278
 4  0.875801 -0.124088  0.071768 -0.188519
 5  0.879000 -0.002539 -0.000013  0.170965
 6  0.063678  0.158350  0.265593  0.370071,
           0         1         2         3
 0  0.838151 -0.523330  0.092727 -0.122597
 1  0.481565  0.785763  0.336851  0.192881
 2 -0.255311 -0.216087  0.935279 -0.115654
 3 -0.020336 -0.249017  0.056448  0.966639)

In [1]:
import pandas as pd